In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, Flatten, Input, Activation, PReLU
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit
import random
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
import shap


# Create Datasets

### Import main dataset

In [6]:
# import excel sheet to pandas dataframe
rows_to_keep = np.arange(323)
df=pd.read_excel(open('../Data/SPS-Commorbiditiesver6.xlsx', 'rb'), sheet_name='Reorganized-with-labels (2)',skiprows = lambda x: x not in rows_to_keep, usecols=range(49))
df

,Past medical history item,1,2,3,7,8,9,10,11,12,...,46,47,48,49,50,51,52,53,54,55
0,(cv) aneurysm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,(cv) aortic stenosis,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,(cv) aortoiliac occlusive disease,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,(cv) atherosclerotic cardiovascular disease,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,(cv) chest tightness or pressure,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,(rheum/immuno) inflammatory arthritis (unspeci...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
318,(rheum/immuno) psoriasis,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
319,(rheum/immuno) rheumatoid arthritis,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
320,(rheum/immuno) Sjogren's syndrome,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Divide into sub dataframes 

In [3]:
df_cv = df.iloc[:13, :]
df_derm = df.iloc[13:19, :]
df_endo = df.iloc[19:38, :]
df_ent = df.iloc[38:43, :]
df_gi = df.iloc[43:63, :]
df_heme = df.iloc[63:75, :]
df_id = df.iloc[75:83, :]
df_msk = df.iloc[83:133, :]
df_nephro = df.iloc[133:155, :]
df_neuro = df.iloc[155:242, :]
df_obgyn = df.iloc[242:254, :]
df_ophth = df.iloc[254:266, :]
df_psyc = df.iloc[266:303, :]
df_resp = df.iloc[303:312, :]
df_rheum = df.iloc[312:321, :]

df_list=[df_cv,df_derm,df_endo,df_ent,df_gi,df_heme,df_id,df_msk,df_nephro,df_neuro,df_obgyn,df_ophth,df_psyc,
        df_resp,df_rheum]
domain_list=["cv","derm","endo","ent","gi","heme","id","msk","nephro","neuro","obgyn","ophth","psyc","resp","rheum"]

### For each symptom domain, perform classification (SPS vs. Non-SPS)

In [4]:
#initialize labels (isSPS)
labels_np=df.to_numpy().transpose()[1:].transpose()[-1].transpose()
labels_np = np.asarray(labels_np).astype('float32')

#initialize cross validation scheme
skf = StratifiedKFold(n_splits=24,random_state=random.randint(0,300),shuffle=True)
rskf = RepeatedStratifiedKFold(n_splits=23, n_repeats=20, random_state=36851234)

score_list=[]

index_num=0

for dataframe in df_list:
    features_np=dataframe.to_numpy().transpose()[1:]
    features_np = np.asarray(features_np).astype('float32')
    print(features_np.shape)
    print("=====================================")
    print("STARTING NEW DOMAIN")
    print(domain_list[index_num])
    print("=====================================")
    total_score=0
    k=0
    for train_index, test_index in rskf.split(features_np, labels_np):
        print(str(k)+"th run")
        print("=====================================")
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = features_np[train_index], features_np[test_index]
        y_train, y_test = labels_np[train_index], labels_np[test_index]

        clf = svm.SVC()
        y_pred = clf.fit(X_train, y_train).predict(X_test)

        score=accuracy_score(y_test, y_pred)
        print(score)
        total_score=total_score+score
        print("=====================================")
        k=k+1

    total_score=total_score/k
    print("Total Score: " + str(total_score))
    score_list.append(total_score)
    index_num=index_num+1

print("=====================================")
print("REPORTING SUMMARY OF SCORES")
print("=====================================")

for i in range(len(score_list)):
    print(domain_list[i]+": "+str(score_list[i]))

(48, 13)
STARTING NEW DOMAIN
cv
0th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24
 26 27 28 29 30 31 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [21 25 32]
0.3333333333333333
1th run
TRAIN: [ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 37 38 39 40 42 43 44 45 46 47] TEST: [ 3 36 41]
0.0
2th run
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 1 34]
0.5
3th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [10 33]
0.5
4th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [13 28]
0.5
5th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20 21 22 2

0.0
65th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [10 32]
0.5
66th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 47] TEST: [27 46]
0.0
67th run
TRAIN: [ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 3 16]
0.0
68th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 41 42 43 44 45 46 47] TEST: [39 40]
0.0
69th run
TRAIN: [ 0  1  2  3  4  5  6  7  8 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 9 10 33]
0.6666666666666666
70th run
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 3

0.0
124th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31 32 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [24 33]
0.5
125th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [14 30]
0.5
126th run
TRAIN: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 4 28]
0.5
127th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 41 42 43 44 45 46 47] TEST: [13 40]
0.5
128th run
TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 0 34]
0.5
129th run
TRAIN: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 3

0.5
191th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 46] TEST: [45 47]
0.0
192th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 45 46 47] TEST: [19 44]
0.5
193th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 39 40 41 42 43 44 45 46 47] TEST: [13 38]
0.0
194th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [12 16]
0.0
195th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 26 27 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [25 28]
0.5
196th run
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 3

241th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [10 30]
0.5
242th run
TRAIN: [ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 40 41 42 43 44 45 46 47] TEST: [ 2 39]
0.0
243th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 45 46 47] TEST: [27 44]
0.5
244th run
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 1 35]
0.0
245th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 44 45 46 47] TEST: [15 43]
0.5
246th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33

...

0.5
170th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 35 36 37 39 40 41 42 43 44 45 46 47] TEST: [34 38]
0.5
171th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [15 30]
0.5
172th run
TRAIN: [ 0  1  2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 40 41 42 43 44 45 46 47] TEST: [ 7 39]
0.5
173th run
TRAIN: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 4 18]
0.5
174th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 45 46 47] TEST: [26 44]
0.5
175th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24
 25 26 27 28 30 31 32 33 3

0.5
246th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 35 36 37 39 40 41 42 43 44 45 46 47] TEST: [34 38]
0.5
247th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31 32 33 34 35 37 38 39 40 41 42 43 44 45 46 47] TEST: [24 36]
0.5
248th run
TRAIN: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 5 22]
0.5
249th run
TRAIN: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 4 26]
0.5
250th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 43 44 45 46 47] TEST: [19 42]
0.5
251th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 3

0.5
311th run
TRAIN: [ 0  1  2  3  4  5  6  8  9 10 11 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 7 12]
0.5
312th run
TRAIN: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 4 27]
0.5
313th run
TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 0 30]
0.5
314th run
TRAIN: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 5 28]
0.5
315th run
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 1 36]
0.5
316th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31 32 3

0.5
368th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 40 41 42 43 45 46 47] TEST: [22 39 44]
0.3333333333333333
369th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [12 17 32]
0.6666666666666666
370th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [15 35]
0.5
371th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24
 25 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [13 26]
0.5
372th run
TRAIN: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 9 19]
0.5
373th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 15 16 17 18 19 20 21 22 23

0.5
420th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46] TEST: [15 47]
0.5
421th run
TRAIN: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 5 14]
0.5
422th run
TRAIN: [ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 8 22]
0.5
423th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 24
 25 26 27 28 29 30 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [23 31]
0.5
424th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 40 41 42 43 44 45 46 47] TEST: [16 39]
0.5
425th run
TRAIN: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 3

### Based on all available symptoms, perform classification (SPS vs. Non-SPS)

In [73]:
all_np=df.to_numpy().transpose()[1:].transpose()
all_np = np.asarray(all_np).astype('float32')

all_features=all_np[0:-1].transpose()
print(all_features.shape)

(48, 321)


In [79]:
total_score=0
k=0
for train_index, test_index in rskf.split(all_features, labels_np):
    print(str(k)+"th run")
    print("=====================================")
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = all_features[train_index], all_features[test_index]
    y_train, y_test = labels_np[train_index], labels_np[test_index]

    clf = svm.SVC()
    y_pred = clf.fit(X_train, y_train).predict(X_test)

    score=accuracy_score(y_test, y_pred)
    print(score)
    total_score=total_score+score
    print("=====================================")
    k=k+1

total_score=total_score/k
print("Total Score: " + str(total_score))

0th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24
 26 27 28 29 30 31 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [21 25 32]
0.6666666666666666
1th run
TRAIN: [ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 37 38 39 40 42 43 44 45 46 47] TEST: [ 3 36 41]
0.3333333333333333
2th run
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 1 34]
0.5
3th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [10 33]
0.5
4th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [13 28]
0.5
5th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20 21 22 23 24
 25 27 28 29

0.5
57th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 46 47] TEST: [12 45]
0.5
58th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24
 25 26 27 28 29 30 31 32 33 34 35 37 38 39 40 41 42 43 44 45 46 47] TEST: [22 36]
0.5
59th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 42 43 44 45 46 47] TEST: [29 41]
0.5
60th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46] TEST: [24 47]
1.0
61th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [13 17]
1.0
62th run
TRAIN: [ 0  1  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 3

0.5
102th run
TRAIN: [ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 2 29]
0.5
103th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31 32 33 34 35 36 38 39 40 41 42 43 44 45 46 47] TEST: [24 37]
1.0
104th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [12 17]
0.5
105th run
TRAIN: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 5 19]
0.5
106th run
TRAIN: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 39 40 41 42 43 44 45 46 47] TEST: [ 9 38]
1.0
107th run
TRAIN: [ 0  1  2  3  4  5  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 3

0.5
166th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 24
 25 26 27 28 29 30 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [23 31]
0.5
167th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 28 29 30 31 32 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [27 33]
0.5
168th run
TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 0 20]
0.5
169th run
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 38 39 40 41 42 43 44 45 46 47] TEST: [ 1 37]
0.5
170th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 35 36 37 39 40 41 42 43 44 45 46 47] TEST: [34 38]
1.0
171th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 31 32 33 3

1.0
217th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46] TEST: [27 47]
0.5
218th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 45 46 47] TEST: [10 44]
0.5
219th run
TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 0 34]
0.5
220th run
TRAIN: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 9 28]
0.5
221th run
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 1 29]
0.5
222th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27 28 29 30 31 32 3

0.5
273th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 33 34 35 36 37 39 40 41 42 43 44 45 46 47] TEST: [32 38]
1.0
274th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46] TEST: [27 47]
0.5
275th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 27 28 29 30 31 32 33 34 35 36 38 39 40 41 42 43 44 45 46 47] TEST: [26 37]
1.0
276th run
TRAIN: [ 1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 0  9 36]
0.6666666666666666
277th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 27 28 29 30 31 32 33 34 35 36 38 39 40 41 42 43 45 46 47] TEST: [26 37 44]
0.6666666666666666
278th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22

0.3333333333333333
324th run
TRAIN: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 4 22]
0.5
325th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 38 39 40 41 42 43 44 46 47] TEST: [37 45]
0.5
326th run
TRAIN: [ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 8 16]
1.0
327th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 44 45 46 47] TEST: [21 43]
0.5
328th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 12 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [10 13]
0.5
329th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20 21 22 24 25
 26 27 28 2

0.5
357th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 44 45 46 47] TEST: [19 43]
1.0
358th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [27 35]
0.5
359th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 46 47] TEST: [25 45]
0.5
360th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [16 33]
0.5
361th run
TRAIN: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [ 5 18]
0.0
362th run
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 3

1.0
415th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24
 26 27 28 29 30 31 32 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [21 25 33]
1.0
416th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [12 27]
0.5
417th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] TEST: [10 29]
0.5
418th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 35 36 37 38 39 40 41 42 43 44 46 47] TEST: [34 45]
0.5
419th run
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 45 46 47] TEST: [ 1 44]
0.5
420th run
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 3